In [21]:
import numpy as np
import matplotlib.pyplot as plt
from numba import njit
import scipy 
import datetime

In [42]:
L = 50
u = 0.001
k1 = 1
k2 = 0.04
eps = 0.5
alpha = 0.001
F0 = 1
m = 1
h = 0.1

In [44]:
@njit
def acc(x,v,t):
    xIntoLeft = np.roll(x,-1)
    xIntoRight = np.roll(x,1)
    xIntoLeft[-1] = 0
    xIntoRight[0] = 0

    return (-x*(2*k1 + k2) + (xIntoRight + xIntoLeft)*k1 + k2*u*t - f(x,v,t) )/m

In [45]:
@njit
def acc2(x,v,t):
    return (-x*(2*k1 + k2) + k2*u*t - f2(x,v,t) )/m

In [46]:
@njit
def f(x,v,t):
     xIntoLeft = np.roll(x,-1)
     xIntoRight = np.roll(x,1)
     xIntoLeft[-1] = 0
     xIntoRight[0] = 0
     return np.where(v==0,
     np.where(np.abs(-x*(2*k1 + k2) + (xIntoRight + xIntoLeft)*k1 + k2*u*t) < F0, -x*(2*k1 + k2) + (xIntoRight + xIntoLeft)*k1 + k2*u*t ,F0*(1-eps) ),
     F0*(1-eps)/(1+alpha*np.abs(v)))
     ### TODO check for v<0

In [47]:
@njit
def f2(x,v,t):
     return np.where(v==0,
     np.where(np.abs(-x*(2*k1 + k2)+ k2*u*t) < F0, -x*(2*k1 + k2) + k2*u*t ,F0*(1-eps) ),
     F0*(1-eps)/(1+alpha*np.abs(v)))
     ### TODO check for v<0

In [48]:
def tenstion(x,v,t):
    return (-x*(2*k1 + k2) + k2*u*t )/m

In [49]:
@njit
def step2(x,v,t):
    k1 = v
    l1 = acc2(x,v,t)

    k2 = v + l1*h/2
    l2 = acc2(x+k1*h/2,v + l1*h/2,t + h/2)

    k3 = v + l2*h/2
    l3 = acc2(x+k2*h/2,v + l2*h/2,t + h/2)

    k4 = v + l3/2
    l4 = acc2(x+k3*h,v + l3*h, t + h)

    xNew = x +  (k1 + 2*k2 + 2*k3 + k4)*h/6
    vNew = v +  (l1 + 2*l2 + 2*l3 + l4)*h/6

    vNew = 0 if vNew < 0 else vNew 
    
    return xNew, vNew,t+h

In [50]:
@njit
def step(x,v,t):
    k1 = v
    l1 = acc(x,v,t)

    k2 = v + l1*h/2
    l2 = acc(x+k1*h/2,v + l1*h/2,t + h/2)

    k3 = v + l2*h/2
    l3 = acc(x+k2*h/2,v + l2*h/2,t + h/2)

    k4 = v + l3/2
    l4 = acc(x+k3*h,v + l3*h, t + h)

    xNew = x +  (k1 + 2*k2 + 2*k3 + k4)*h/6
    vNew = v +  (l1 + 2*l2 + 2*l3 + l4)*h/6

    vNew = np.where(vNew<0, 0, vNew)
    
    return xNew, vNew,t+h

In [51]:
def run(x,v,t,steps=1_000_000):
    xdata = np.zeros((steps,L),dtype=np.float32)
    vdata = np.zeros((steps,L),dtype=np.float32)
    for i in range(steps):
        x,v,t = step(x,v,t)
        xdata[i] = x
        vdata[i] = v

    slipping = np.any(vdata>0,axis=1)

    slippingR = np.roll(slipping,+1)
    slippingL = np.roll(slipping,-1)

    slippingR[0] = False
    slippingL[-1] = False 

    ends  = np.logical_xor( slipping ,(slipping *slippingL))
    starts  = np.logical_xor( slipping ,(slipping *slippingR))

    totalX = np.sum(xdata,axis = 1)

    eventsSize = totalX[np.argwhere(ends)] - totalX[np.argwhere(starts)]
    eventsBlocks = np.sum(xdata[ends]!=xdata[starts],axis=1).reshape(-1,1)

    time = str(datetime.datetime.now())

    np.save("xdata-"+time,xdata)
    np.save("vdata-"+time,vdata)



    return x,v,t,eventsSize,eventsBlocks


In [55]:
x = 0.1*np.random.uniform(-1,1,L).astype(dtype=np.float32) 
v = np.zeros(L,dtype=np.float32)
t = 0

In [56]:
x,v,t,eventsSize,eventsBlock = run(x,v,t,1_000_000)

In [ ]:
parts = 2000
steps = 1_000_000
eventsSizes = np.array([]).reshape((0,1))
eventsBlocks = np.array([]).reshape((0,1))
for p in range(parts):
    x,v,t,eventsSize,eventsBlock = run(x,v,t,steps)
    eventsSizes = np.concatenate((eventsSizes,eventsSize))
    eventsBlocks = np.concatenate((eventsBlocks,eventsBlock))
    print(p)

np.save("eventsSizes",eventsSizes)
np.save("eventsBlocks",eventsBlocks)

    

In [ ]:
dists = []
for i in range(L):
    dists.append(eventsSizes[eventsBlocks==i])

In [ ]:
plt.hist(eventsSizes[eventsBlocks==1],bins=300)

In [ ]:
plt.hist(eventsSizes)

In [ ]:
freq,size = np.histogram(eventsSizes)

In [ ]:
#slope , intercept, rvalue ,_,_= scipy.stats.linregress(np.log(size[:-1]),np.log(freq))

plt.title("frequency vs size")
plt.xlabel("event size")
plt.ylabel("event frequency")
plt.scatter(np.log(size[:-1]),np.log(freq))
#plt.plot([np.log(size[0]),np.log(size[-2])],[intercept + slope.astype(np.float32)*np.log(size[0]), intercept + slope.astype(np.float32)*np.log(size[-2])])
#plt.text(-1,10,s = f"slope = {np.round(slope,3)}, intercept = {np.round(intercept,3)}, rval = {np.round(rvalue,3)}")
plt.savefig("plot.png")

In [ ]:
x = 0
v = 0
t = 0
steps = 100_000
xData = np.zeros(steps)
vData = np.zeros(steps)

In [ ]:
for i in range(steps):
    xData[i] = x
    vData[i] = v
    x,v,t = step2(x,v,t)


In [ ]:
plt.plot(np.arange(steps),xData)

In [ ]:
plt.plot(np.arange(steps),vData)

In [ ]:
tensionData= np.zeros(steps)
for i in range(steps):
    tensionData[i] = tenstion(xData[i],vData[i],i*h)

In [ ]:
fData= np.zeros(steps)
for i in range(steps):
    fData[i] = f2(xData[i],vData[i],i*h)

In [ ]:
plt.plot(np.arange(9900,10100),xData[9900:10100])

In [ ]:
plt.plot(np.arange(9900,10100),vData[9900:10100])

In [ ]:
plt.plot(np.arange(9900,10100),fData[9900:10100])

In [ ]:
plt.plot(np.arange(9900,10100),tensionData[9900:10100])

In [ ]:
plt.plot(np.arange(steps),tensionData)